### Ler Arquivo .env para Pegar Credenciais do Dynamics

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.getenv('DYNAMICS_CLIENT_ID')
client_secret = os.getenv('DYNAMICS_CLIENT_SECRET')
tenant_id = os.getenv('DYNAMICS_TENANT_ID')
dynamics_region = os.getenv('DYNAMICS_REGION')
dynamics_instance = os.getenv('DYNAMICS_INSTANCE')


print(f"Client ID: {client_id}")

Client ID: 645db0f9-bcdc-4306-b54d-1c3fc3f22942


### Obter Token para o Dynamics usando msal

In [7]:
import msal

# BEGIN: Obter Token para o Dynamics
authority_url = f"https://login.microsoftonline.com/{tenant_id}"
app = msal.ConfidentialClientApplication(
    client_id,
    authority=authority_url,
    client_credential=client_secret,
)

token_response = app.acquire_token_for_client(scopes=[f"https://{dynamics_instance}.{dynamics_region}.dynamics.com/.default"])

if "access_token" in token_response:
    access_token = token_response["access_token"]
    print("Access token obtained successfully.")
else:
    print("Failed to obtain access token.")
    print(token_response)
# END: Obter Token para o Dynamics

Access token obtained successfully.


### Obter dados do dynamics

In [12]:
import requests

# BEGIN: Obter dados do Dynamics
dynamics_api_url = f"https://{dynamics_instance}.{dynamics_region}.dynamics.com/api/data/v9.0/contacts?$select=fullname"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "OData-MaxVersion": "4.0",
    "OData-Version": "4.0",
    "Prefer": 'odata.include-annotations="*"'
}

response = requests.get(dynamics_api_url, headers=headers)

if response.status_code == 200:
    dynamics_data = response.json()["value"]
    print("Dados obtidos com sucesso.")
    print(dynamics_data)
else:
    print("Falha ao obter dados.")
    print(response.status_code, response.text)
# END: Obter dados do Dynamics

Dados obtidos com sucesso.
[{'@odata.etag': 'W/"2273635"', 'fullname': 'Jose Andre Filho', 'contactid': '51d7826d-ba70-ef11-a670-000d3a34929e'}]


In [14]:
import pandas as pd

# Converter dynamics_data para DataFrame
df_dynamics = pd.DataFrame(dynamics_data)

# Selecionar somente as colunas fullname e contactid
df_dynamics = df_dynamics[['fullname', 'contactid']]

# Gravar o DataFrame em um arquivo Parquet
df_dynamics.to_parquet('dynamics_data.parquet', index=False)

print("Dados gravados com sucesso no arquivo dynamics_data.parquet")

Dados gravados com sucesso no arquivo dynamics_data.parquet
